### Import libraries

In [1]:
%pip install spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-04-15 13:50:35.360799: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll


In [2]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
import pandas as pd
import os
nlp = spacy.load('en_core_web_sm')

### Preprocessing data
In this part we gonna clean data, split them into inputs and targets tensor, build a tokenizer dictionary and turn sentences into sequences.
The target tensor has a bunch of list with a length of unique title list.

In [3]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

### Inputs

In [4]:
inputs, targets = [], []
classes = []
intent_doc = {}

In [5]:
for root, dirs, files in os.walk('data'):
    print(f"root={root}, dirs={dirs}, files={files}")
    for f in files:
        path = os.path.join(root, f)
        dataframe = pd.read_csv(path)

        for index, line in dataframe.iterrows():
            intent = line['intent']
            if intent not in intent_doc:
                classes.append(intent)
                intent_doc[intent] = []
            if 'request' in line:
                inputs.append(preprocessing(line['request']))
                targets.append(intent)
            if 'response' in line:
                intent_doc[intent].append(line['response'])


root=data, dirs=[], files=['Clever_request.csv', 'Clever_response.csv', 'CourtesyGoodBye_request.csv', 'CourtesyGoodBye_response.csv', 'CourtesyGreetingResponse_request.csv', 'CourtesyGreetingResponse_response.csv', 'CourtesyGreeting_request.csv', 'CourtesyGreeting_response.csv', 'covid_19_request.csv', 'covid_19_response.csv', 'CurrentHumanQuery_request.csv', 'CurrentHumanQuery_response.csv', 'GoodBye_request.csv', 'GoodBye_response.csv', 'Gossip_request.csv', 'Gossip_response.csv', 'GreetingResponse_request.csv', 'GreetingResponse_response.csv', 'Greeting_request.csv', 'Greeting_response.csv', 'Jokes_request.csv', 'Jokes_response.csv', 'NameQuery_request.csv', 'NameQuery_response.csv', 'NotTalking2U_request.csv', 'NotTalking2U_response.csv', 'PodBayDoorResponse_request.csv', 'PodBayDoorResponse_response.csv', 'PodBayDoor_request.csv', 'PodBayDoor_response.csv', 'RealNameQuery_request.csv', 'RealNameQuery_response.csv', 'SelfAware_request.csv', 'SelfAware_response.csv', 'Shutup_reques

In [6]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [7]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [8]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (212, 9) and output shape: (212, 27)


### Build the model

In [9]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [10]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 512)         94208     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               656384    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 27)                3483      
Total params: 786,971
Trainable params: 786,971
Non-trainable params: 0
_________________________________________________________________


In [11]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
7/7 [==============================] - 8s 15ms/step - loss: 3.3278 - accuracy: 0.0541
Epoch 2/50
7/7 [==============================] - 0s 12ms/step - loss: 2.8654 - accuracy: 0.2125
Epoch 3/50
7/7 [==============================] - 0s 11ms/step - loss: 1.6809 - accuracy: 0.5415
Epoch 4/50
7/7 [==============================] - 0s 10ms/step - loss: 0.9260 - accuracy: 0.7326
Epoch 5/50
7/7 [==============================] - 0s 10ms/step - loss: 0.4793 - accuracy: 0.8414
Epoch 6/50
7/7 [==============================] - 0s 10ms/step - loss: 0.3145 - accuracy: 0.8935
Epoch 7/50
7/7 [==============================] - 0s 9ms/step - loss: 0.1999 - accuracy: 0.9439
Epoch 8/50
7/7 [==============================] - 0s 10ms/step - loss: 0.1890 - accuracy: 0.9403
Epoch 9/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0522 - accuracy: 0.9851
Epoch 10/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0971 - accuracy: 0.9668
Epoch 11/50
7/7 [=============

### Save the model

In [14]:
import pickle
model.save('deployment/chatbot.h5')
with open('deployment/tokenizer.json', 'w') as f:
    f.write(tokenizer.to_json())
pickle.dump(intent_doc, open('deployment/intent_doc.pickle','wb'))
pickle.dump(trg_index_word, open('deployment/trg_index_word.pickle','wb'))


In [12]:
#Define API

In [13]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]


In [14]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route("/")
def hello():
    return render_template('chat.html')

@app.route("/ask", methods=['POST'])
def ask():
    message = request.form['messageText']
    print(message)
    while True:
        if message == "quit":
            exit()
        else:
            bot_response, intent_type = response(message)
            print(bot_response)
            return jsonify({'status':'OK','answer':bot_response, 'intent_type':intent_type})
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
